<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

## Before we begin.. after reading the description.txt

In [64]:
# Load the data
house = pd.read_csv('./housing.csv')

In [72]:
house.shape

(1460, 81)

In [5]:
#Check Cols and check Descr to see for anything missing
house.columns

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [65]:
#Check Null value cols
house.columns[house.isna().any()].tolist()

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [66]:
#Check LotFrontage first - Desc: LotFrontage: Linear feet of street connected to property
house[house['LotFrontage'].isnull()].head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
12,13,20,RL,NaN,12968,Pave,NaN,IR2,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,144000
14,15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,5,2008,WD,Normal,157000
16,17,20,RL,NaN,11241,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,700,3,2010,WD,Normal,149000
24,25,20,RL,NaN,8246,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2010,WD,Normal,154000


In [70]:
house[house['LotFrontage'] == 0]
### SHALL REPLACE NaN with 0 ###

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [71]:
## NEXT: Check Alley - Desc: 
# Alley: Type of alley access to property
#
#       Grvl	Gravel
#       Pave	Paved
#       NA 	No alley access

house.Alley.value_counts() ## This means i should have house.shape (1460,81) : 1460 - 91 = 1396 NaNs for Alley.

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [76]:
## 1396 Check.
house.Alley.isnull().sum()

1369

### Getting suspicious about NA..

In [ ]:
## Check MasVnrType for None as NA? 
# MasVnrType: Masonry veneer type
#
#       BrkCmn	Brick Common
#       BrkFace	Brick Face
#       CBlock	Cinder Block
#       None	None
#       Stone	Stone

In [78]:
house.MasVnrType.value_counts() ## This means i should have house.shape (1460,81) : 1460 - 1452 = 8 NaNs for None.

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [79]:
house[house['MasVnrType'].isnull()].loc[:, 'MasVnrType':] #Just see the col slice, if not it wont display

,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
234,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,625,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,216500
529,NaN,NaN,Gd,TA,PConc,TA,TA,No,Rec,1219,...,0,NaN,NaN,NaN,0,3,2007,WD,Alloca,200624
650,NaN,NaN,Gd,TA,PConc,Gd,TA,No,Unf,0,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,205950
936,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,833,...,0,NaN,NaN,NaN,0,8,2009,WD,Normal,184900
973,NaN,NaN,Gd,TA,PConc,Gd,TA,No,Unf,0,...,0,NaN,NaN,NaN,0,12,2008,New,Partial,182000
977,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,1106,...,0,NaN,NaN,NaN,0,11,2007,New,Partial,199900
1243,NaN,NaN,Ex,TA,PConc,Ex,Gd,Gd,GLQ,1386,...,0,NaN,NaN,NaN,0,9,2006,New,Partial,465000
1278,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,804,...,0,NaN,NaN,NaN,0,3,2008,WD,Normal,237000


In [80]:
house.MasVnrType.isnull().sum()
## 8 Check.. but that was for 'None' but not explicitly 'NA'

8

In [81]:
## LAST CHECK for NA.

# BsmtQual: Evaluates the height of the basement
#
#       Ex	Excellent (100+ inches)	
#       Gd	Good (90-99 inches)
#       TA	Typical (80-89 inches)
#       Fa	Fair (70-79 inches)
#       Po	Poor (<70 inches
#       NA	No Basement

In [88]:
house.BsmtQual.value_counts() ## No 'Po'

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [91]:
## Check to see who has 'Po' basement with BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1
house[(house['BsmtQual'] == 'Po') | \
      (house['BsmtCond'] == 'Po') | \
      (house['BsmtExposure'] == 'Po') | \
      (house['BsmtFinType1'] == 'Po')].loc[:, 'BsmtQual':] #Just see the col slice, if not it wont display

## The rows with 'Po' has no NaNs for the basement features.

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
375,Fa,Po,Gd,BLQ,350,Unf,0,333,683,GasA,...,0,NaN,NaN,NaN,0,3,2009,WD,Normal,61000
398,Fa,Po,No,Unf,0,Unf,0,961,961,GasA,...,0,NaN,NaN,NaN,0,11,2007,WD,Abnorml,67000


In [94]:
## So.. BsmtQual has 1460 - 1423 = 37 NaNs
house.BsmtQual.isnull().sum()
# Check on 37.

37

## NaNs replace 'NA's in the cols. (Many cols affected) Not good. Restart.


<p>From pandas read_csv</p>
<p><strong>na_values</strong>&nbsp;: scalar, str, list-like, or dict, default None</p>
<blockquote>
<div>
<p>Additional strings to recognize as NA/NaN. If dict passed, specific per-column NA values. By default the following values are interpreted as NaN: &lsquo;&rsquo;, &lsquo;#N/A&rsquo;, &lsquo;#N/A N/A&rsquo;, &lsquo;#NA&rsquo;, &lsquo;-1.#IND&rsquo;, &lsquo;-1.#QNAN&rsquo;, &lsquo;-NaN&rsquo;, &lsquo;-nan&rsquo;, &lsquo;1.#IND&rsquo;, &lsquo;1.#QNAN&rsquo;, &lsquo;N/A&rsquo;, &lsquo;<span style="background-color: #00ff00;">NA</span>&rsquo;, &lsquo;NULL&rsquo;, &lsquo;NaN&rsquo;, &lsquo;n/a&rsquo;, &lsquo;nan&rsquo;, &lsquo;null&rsquo;.</p>
</div>
</blockquote>
<p><strong>keep_default_na</strong>&nbsp;: bool, default True</p>
<blockquote>
<div>
<p>Whether or not to include the default NaN values when parsing the data. Depending on whether&nbsp;<cite>na_values</cite>&nbsp;is passed in, the behavior is as follows:</p>
<ul class="simple">
<li>If&nbsp;<cite>keep_default_na</cite>&nbsp;is True, and&nbsp;<cite>na_values</cite>&nbsp;are specified,&nbsp;<cite>na_values</cite>&nbsp;is appended to the default NaN values used for parsing.</li>
<li>If&nbsp;<cite>keep_default_na</cite>&nbsp;is True, and&nbsp;<cite>na_values</cite>&nbsp;are not specified, only the default NaN values are used for parsing.</li>
<li>If&nbsp;<cite>keep_default_na</cite>&nbsp;is False, and&nbsp;<cite>na_values</cite>&nbsp;are specified, only the NaN values specified&nbsp;<cite>na_values</cite>&nbsp;are used for parsing.</li>
<li><span style="background-color: #00ff00;">If&nbsp;<cite>keep_default_na</cite>&nbsp;is False, and&nbsp;<cite>na_values</cite>&nbsp;are not specified, no strings will be parsed as NaN.</span></li>
</ul>
<p>Note that if&nbsp;<cite>na_filter</cite>&nbsp;is passed in as False, the&nbsp;<cite>keep_default_na</cite>&nbsp;and&nbsp;<cite>na_values</cite>&nbsp;parameters will be ignored.</p>
</div>
</blockquote>

### Lucky break.

In [95]:
house2 = pd.read_csv('./housing.csv', keep_default_na=False)

In [96]:
## Lets double check on Alley's 'NA' -> Id == 1
house[house['Id'] == 1]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [97]:
house2[house2['Id'] == 1]
## Success!

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500


In [98]:
house2.columns[house2.isna().any()].tolist()
## No NaNs in the DF. ^_^

[]

In [99]:
house2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null object
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-

In [101]:
house2.select_dtypes(include=['object']).head(10)

,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,65,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,2003,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
1,RL,80,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,1976,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
2,RL,68,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,2001,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
3,RL,60,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,1998,Unf,TA,TA,Y,NA,NA,NA,WD,Abnorml
4,RL,84,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,2000,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
5,RL,85,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,...,1993,Unf,TA,TA,Y,NA,MnPrv,Shed,WD,Normal
6,RL,75,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,...,2004,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
7,RL,NA,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,...,1973,RFn,TA,TA,Y,NA,NA,Shed,WD,Normal
8,RM,51,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,...,1931,Unf,Fa,TA,Y,NA,NA,NA,WD,Abnorml
9,RL,50,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,...,1939,RFn,Gd,TA,Y,NA,NA,NA,WD,Normal


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [4]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [5]:
# A: